In [1]:
import cv2 as cv
import numpy as np
import dlib

# 加載Dlib的人臉檢測器
detector = dlib.get_frontal_face_detector()

input_video = './video/Alec_Baldwin.mp4'
output_video = './video/Alec_Baldwin_opencv_video_effects.mp4'

cap = cv.VideoCapture(input_video)

width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv.CAP_PROP_FPS))

fourcc = cv.VideoWriter_fourcc(*'mp4v')
out = cv.VideoWriter(output_video, fourcc, fps, (width, height))

effect_name = ""
effect_frames = 120 # 每個效果持續的幀數
frame_count = 0
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

# 在特效迴圈開始前加"OPENCV VIDEO EFFECTS"和"by Stella"字卡
start_frames_count = int(fps * 3)
start_text = "OPENCV VIDEO EFFECTS"
start_font = cv.FONT_HERSHEY_SIMPLEX
start_font_scale = 2
start_color = (255, 255, 255)
start_thickness = 3

text_size, _ = cv.getTextSize(start_text, start_font, start_font_scale, start_thickness)
text_width, text_height = text_size
text_x = (frame_width - text_width) // 2
text_y = (frame_height - text_height) // 2

author_text = "by Stella"
author_font_scale = 1
author_thickness = 2

author_text_size, _ = cv.getTextSize(author_text, start_font, author_font_scale, author_thickness)
author_text_width, author_text_height = author_text_size
author_text_x = (frame_width - author_text_width) // 2
author_text_y = text_y + text_height + author_text_height + 10  # 10 pixels below start_text

for i in range(start_frames_count):
    start_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
    cv.putText(start_frame, start_text, (text_x, text_y), start_font, start_font_scale, start_color, start_thickness, cv.LINE_AA)
    cv.putText(start_frame, author_text, (author_text_x, author_text_y), start_font, author_font_scale, start_color, author_thickness, cv.LINE_AA)
    out.write(start_frame)


def adjust_gamma(image, gamma):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv.LUT(image, table)

    
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    effect = frame_count // effect_frames

    if effect == 0:  # 正常效果
        effect_name = "Normal"
    elif effect == 1:  # 灰階效果
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frame = cv.cvtColor(frame, cv.COLOR_GRAY2BGR)
        effect_name = "Grayscale"
    elif effect == 2:  # 高斯模糊效果
        frame = cv.GaussianBlur(frame, (15, 15), 0)
        effect_name = "Gaussian Blur"
    elif effect == 3:  # Canny邊緣檢測效果
        frame = cv.Canny(frame, 100, 200)
        frame = cv.cvtColor(frame, cv.COLOR_GRAY2BGR)
        effect_name = "Canny Edges"
    elif effect == 4:  # 反色效果
        frame = 255 - frame
        effect_name = "Inverted"
    elif effect == 5:  # 棕褐色效果
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        frame = frame.astype('float32')
        frame = frame * (0.272, 0.534, 0.131)
        frame = cv.normalize(frame, None, 0, 255, cv.NORM_MINMAX)
        frame = frame.astype('uint8')
        effect_name = "Sepia"
    elif effect == 6:  # 亮度效果
        frame = cv.convertScaleAbs(frame, alpha=1, beta=50)
        effect_name = "Brightness"
    elif effect == 7:  # 對比度效果
        frame = cv.convertScaleAbs(frame, alpha=1.5, beta=0)
        effect_name = "Contrast"
    elif effect == 8:  # 旋轉180度效果
        M = cv.getRotationMatrix2D((width // 2, height // 2), 180, 1.0)
        frame = cv.warpAffine(frame, M, (width, height))
        effect_name = "Rotate 180"
    elif effect == 9:  # 色彩反轉效果
        frame = cv.bitwise_not(frame)
        effect_name = "Color Inversion"
    elif effect == 10:  # 調整尺寸
        frame = cv.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv.INTER_AREA)
        effect_name = "Resize"
    
    elif effect == 11: #人臉辨識
        effect_name = "Face Detection"
        # 將圖像從BGR轉換為RGB(因為Dlib使用的是RGB)
        image_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        # 使用 Dlib 检测器检测图像中的人脸
        faces = detector(image_rgb)
        # 在檢測到的人臉上繪製矩形框
        for face in faces:
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    elif effect == 12:  # 閾值處理
        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        _, threshold_frame = cv.threshold(gray_frame, 128, 255, cv.THRESH_BINARY)
        frame = cv.cvtColor(threshold_frame, cv.COLOR_GRAY2BGR)
        effect_name = "Threshold"
    elif effect == 13:  # 卡通化
        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        gray_frame = cv.medianBlur(gray_frame, 5)
        edges_frame = cv.adaptiveThreshold(gray_frame, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 9, 9)
        color_frame = cv.bilateralFilter(frame, 9, 250, 250)
        frame = cv.bitwise_and(color_frame, color_frame, mask=edges_frame)
        effect_name = "Cartoon"
    elif effect == 14:  # 素描
        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        gray_frame = cv.GaussianBlur(gray_frame, (13, 13), 0)
        edges_frame = cv.Laplacian(gray_frame, cv.CV_8U, ksize=5)
        _, threshold_frame = cv.threshold(edges_frame, 100, 255, cv.THRESH_BINARY_INV)
        frame = cv.cvtColor(threshold_frame, cv.COLOR_GRAY2BGR)
        effect_name = "Sketch"
    elif effect == 15:  # 中值模糊
        effect_name = 'Median Blur'
        frame = cv.medianBlur(frame, ksize=5)  # ksize必须是奇数
    elif effect == 16: #雙邊濾波
        effect_name = 'Bilateral Filter'
        frame = cv.bilateralFilter(frame, d=9, sigmaColor=75, sigmaSpace=75)   
    elif effect == 17: #膨脹
        effect_name = 'Dilation'
        kernel = np.ones((5, 5), np.uint8)
        frame = cv.dilate(frame, kernel, iterations=1)
    elif effect == 18: #腐蝕
        effect_name = 'Erosion'
        kernel = np.ones((5, 5), np.uint8)
        frame = cv.erode(frame, kernel, iterations=1)       
    elif effect == 19:  # 水平翻轉效果
        frame = cv.flip(frame, 1)
        effect_name = "Horizontal Flip"
    elif effect == 20:  # 垂直翻轉效果
        frame = cv.flip(frame, 0)
        effect_name = "Vertical Flip"
    
    elif effect == 21: #4 in 1 + Gamma
        effect_name = '4 in 1 + Gamma Correction'
        gamma_values = [0.5, 1.5, 2.5, 3.5]
        h, w = frame.shape[:2]
        half_h, half_w = h // 2, w // 2

        # 處理影片的四個區域
        frame[:half_h, :half_w] = adjust_gamma(frame[:half_h, :half_w], gamma_values[0])
        frame[:half_h, half_w:] = adjust_gamma(frame[:half_h, half_w:], gamma_values[1])
        frame[half_h:, :half_w] = adjust_gamma(frame[half_h:, :half_w], gamma_values[2])
        frame[half_h:, half_w:] = adjust_gamma(frame[half_h:, half_w:], gamma_values[3])

        # 在每個區域顯示對應的Gamma值
        font = cv.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        color = (0, 255, 0)
        thickness = 2

        for i, gamma_value in enumerate(gamma_values):
            text = f'Gamma: {gamma_value}'
            x_offset = half_w * (i % 2)
            y_offset = half_h * (i // 2)
            x, y = 30 + x_offset, 50 + y_offset
            cv.putText(frame, text, (x, y), font, font_scale, color, thickness)
    
    elif effect == 22:  # 波形扭曲
        effect_name = "Wave Distortion"
        rows, cols, _ = frame.shape
        frame_output = np.zeros_like(frame)

        for y in range(rows):
            for x in range(cols):
                offset_x = int(25.0 * np.sin(2 * 3.14 * y / 180))
                offset_y = int(25.0 * np.sin(2 * 3.14 * x / 180))
                new_x = x + offset_x
                new_y = y + offset_y

                if 0 <= new_x < cols and 0 <= new_y < rows:
                    frame_output[y, x] = frame[new_y, new_x]
                else:
                    frame_output[y, x] = 0

        frame = frame_output

    # 在左上角加效果名稱
    text_width, text_height = cv.getTextSize(effect_name, cv.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
    cv.rectangle(frame, (0, 0), (text_width + 10, text_height + 10), (0, 0, 0), -1)
    cv.putText(frame, effect_name, (5, text_height + 5), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # 將處理過的效果寫入輸出視頻
    out.write(frame)
    frame_count += 1


# 在特效迴圈結束後加"THE END"字卡
end_frames_count = int(fps * 3)  # 在影片結束時顯示"THE END"3秒
end_text = "THE END"
end_font = cv.FONT_HERSHEY_SIMPLEX
end_font_scale = 2
end_color = (255, 255, 255)
end_thickness = 3

text_size, _ = cv.getTextSize(end_text, end_font, end_font_scale, end_thickness)
text_width, text_height = text_size
text_x = (frame_width - text_width) // 2
text_y = (frame_height - text_height) // 2

for _ in range(end_frames_count):
    black_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
    cv.putText(black_frame, end_text, (text_x, text_y), end_font, end_font_scale, end_color, end_thickness, cv.LINE_AA)
    out.write(black_frame)

# 釋放資源
cap.release()
out.release()
cv.destroyAllWindows()